# YOUR PROJECT TITLE

> **Note the following:** 
> 1. This is *not* meant to be an example of an actual **data analysis project**, just an example of how to structure such a project.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `dataproject.py` file includes a function which can be used multiple times in this notebook.

Imports and set magics:

In [56]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
#from matplotlib_venn import venn2

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# user written modules
import dataproject


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Read and clean data

Import your data, either through an API or manually, and load it. 

In [57]:
filename = 'C:/Users/Bruger/OneDrive/Skrivebord/MPK100.xlsx' # open the file and have a look at it
pd.read_excel(filename).head(5)

,Government bond yields by country and time,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,Units: Per cent per annum,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,2022.0,2023.0
2,Denmark,1.7,1.3,0.7,0.3,0.5,0.5,0.1,0.3,-0.1,3.4,3.0
3,Germany,1.6,1.2,0.5,0.1,0.3,0.4,0.1,0.3,-0.5,3.0,2.8
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
bond = pd.read_excel(filename, skiprows=2)
bond.head(5)

,Unnamed: 0,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Denmark,1.7,1.3,0.7,0.3,0.5,0.5,0.1,0.3,-0.1,3.4,3.0
1,Germany,1.6,1.2,0.5,0.1,0.3,0.4,0.1,0.3,-0.5,3.0,2.8
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Source: OECD.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
# These rows have to go: '0' '4'
drop_these = ['Unnammed: ' + str(num) for num in range(0)] # use list comprehension to create list of columns
print(drop_these)

bond.drop(drop_these, axis=1, inplace=True) # axis = 4 -> columns, inplace=True -> changed, no copy made
bond.head(2)

[]


,Unnamed: 0,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Denmark,1.7,1.3,0.7,0.3,0.5,0.5,0.1,0.3,-0.1,3.4,3.0
1,Germany,1.6,1.2,0.5,0.1,0.3,0.4,0.1,0.3,-0.5,3.0,2.8


In [60]:
# Assuming 'bond' is your DataFrame
rows_to_drop = [2, 3]  # Define the rows you want to drop

# Drop the specified rows
bond.drop(rows_to_drop, inplace=True)

# Display the modified DataFrame
print(bond.head())

  Unnamed: 0  2013  2014  2015  2016  2017  2018  2019  2020  2021  2022  2023
0    Denmark   1.7   1.3   0.7   0.3   0.5   0.5   0.1   0.3  -0.1   3.4   3.0
1    Germany   1.6   1.2   0.5   0.1   0.3   0.4   0.1   0.3  -0.5   3.0   2.8


In [69]:
bond.rename(columns = {'Unnamed: 0':'Country'}, inplace=True)
bond.head(2)

,Country,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Denmark,1.7,1.3,0.7,0.3,0.5,0.5,0.1,0.3,-0.1,3.4,3.0
1,Germany,1.6,1.2,0.5,0.1,0.3,0.4,0.1,0.3,-0.5,3.0,2.8


In [70]:
col_dict = {}
for i in range(2013, 2023+1): # range goes from 2013 to but not including 2018
    col_dict[str(i)] = f'bond{i}'

col_dict = {str(i) : f'bond{i}' for i in range(2013,2023+1)}
col_dict

{'2013': 'bond2013',
 '2014': 'bond2014',
 '2015': 'bond2015',
 '2016': 'bond2016',
 '2017': 'bond2017',
 '2018': 'bond2018',
 '2019': 'bond2019',
 '2020': 'bond2020',
 '2021': 'bond2021',
 '2022': 'bond2022',
 '2023': 'bond2023'}

In [71]:
bond.rename(columns = col_dict, inplace=True)
bond.head(10)

,Country,bond2013,bond2014,bond2015,bond2016,bond2017,bond2018,bond2019,bond2020,bond2021,bond2022,bond2023
0,Denmark,1.7,1.3,0.7,0.3,0.5,0.5,0.1,0.3,-0.1,3.4,3.0
1,Germany,1.6,1.2,0.5,0.1,0.3,0.4,0.1,0.3,-0.5,3.0,2.8


## Explore each data set

In order to be able to **explore the raw data**, you may provide **static** and **interactive plots** to show important developments 

**Interactive plot** :

In [ ]:
def plot_func():
    # Function that operates on data set
    pass

widgets.interact(plot_func, 
    # Let the widget interact with data through plot_func()    
); 


interactive(children=(Output(),), _dom_classes=('widget-interact',))

Explain what you see when moving elements of the interactive plot around. 

# Merge data sets

Now you create combinations of your loaded data sets. Remember the illustration of a (inner) **merge**:

In [ ]:
plt.figure(figsize=(15,7))
v = venn2(subsets = (4, 4, 10), set_labels = ('Data X', 'Data Y'))
v.get_label_by_id('100').set_text('dropped')
v.get_label_by_id('010').set_text('dropped' )
v.get_label_by_id('110').set_text('included')
plt.show()

NameError: name 'venn2' is not defined

<Figure size 1500x700 with 0 Axes>

Here we are dropping elements from both data set X and data set Y. A left join would keep all observations in data X intact and subset only from Y. 

Make sure that your resulting data sets have the correct number of rows and columns. That is, be clear about which observations are thrown away. 

**Note:** Don't make Venn diagrams in your own data project. It is just for exposition. 

# Analysis

To get a quick overview of the data, we show some **summary statistics** on a meaningful aggregation. 

MAKE FURTHER ANALYSIS. EXPLAIN THE CODE BRIEFLY AND SUMMARIZE THE RESULTS.

# Conclusion

ADD CONCISE CONLUSION.